In [1]:
import os
from typing import Any, Dict, Iterator, List, Optional, Sequence, Tuple
import immutabledict
import matplotlib.pyplot as plt
import tensorflow as tf
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd

if not tf.executing_eagerly():
    tf.compat.v1.enable_eager_execution()

from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset import v2
from waymo_open_dataset.protos import camera_segmentation_metrics_pb2 as metrics_pb2
from waymo_open_dataset.protos import camera_segmentation_submission_pb2 as submission_pb2
from waymo_open_dataset.wdl_limited.camera_segmentation import camera_segmentation_metrics
from waymo_open_dataset.utils import camera_segmentation_utils

2023-05-21 23:16:24.725820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 23:16:24.848410: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 23:16:25.344072: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/f

In [2]:
def getFramesList(txtFile, start_context, context_with_issue):
    context_name_list = []
    frames_dict = {}
    list_ = open(txtFile).read().split()
    start_fetching = False
    for line in list_:
        frames_line = line.split(',')
        context_name, time_stamp = frames_line
        if(context_name == start_context):
            start_fetching = True
        if(not start_fetching):
            continue
        if not(context_name in context_name_list) and not(context_name in context_with_issue):
            context_name_list.append(context_name)
        if not(context_name in frames_dict):
            frames_dict[context_name] = [time_stamp]
        else:
            frames_dict[context_name].append(time_stamp)

    for context_name in frames_dict:
        time_stamp_list = frames_dict[context_name]
        frames_dict[context_name] = sorted(time_stamp_list)
    #     print("======", context_name)
    #     print(len(frames_dict[context_name]))
    #     for time in frames_dict[context_name]:
    #         print(time)
    return context_name_list, frames_dict

#@title Submission helper functions

def _make_submission_proto(
) -> submission_pb2.CameraSegmentationSubmission:
    """Makes a submission proto to store predictions for one shard."""
    submission = submission_pb2.CameraSegmentationSubmission()
    submission.account_name = 'kaiwenjon23@gmail.com'
    submission.unique_method_name = 'ViP-DeepLab on single views, then apply image stitching.'
    submission.authors.extend(['Kevin Chuang', 'Yu-An Su', 'Vincent Chen'])
    submission.affiliation = 'University of Illinois at Urbana Champaign'
    submission.description = 'Fine-tune models from ViP-DeepLab on waymo-open-dataset. Then stitch images from five cameras to propagate instance IDs.'
    submission.method_link = 'https://github.com/google-research/deeplab2/'
    submission.frame_dt = 1
    submission.runtime_ms = 1000
    return submission

camera_names = [
    "SIDE_LEFT",
    "FRONT_LEFT",
    "FRONT",
    "FRONT_RIGHT",
    "SIDE_RIGHT"
]
camera_left_to_right_order = [open_dataset.CameraName.SIDE_LEFT,
                              open_dataset.CameraName.FRONT_LEFT,
                              open_dataset.CameraName.FRONT,
                              open_dataset.CameraName.FRONT_RIGHT,
                              open_dataset.CameraName.SIDE_RIGHT]
def my_generate_predictions_for_one_test_shard(
        submission: submission_pb2.CameraSegmentationSubmission,
        context_name : str,
        timestamps: [str]
    ) -> None:
    
    panoptic_label_divisor = 1000
    for timestamp in timestamps:
        for i, camera_name in enumerate(camera_names):
            panoptic_pred = np.zeros((100, 100, 1), dtype=np.int32)
            panoptic_pred_rgb = np.zeros((100, 100, 3)) # cv2.imread(f"{context_name}/{timetstamp}/{camera_name}")
            panoptic_pred[:, :, 0] = panoptic_pred_rgb[:, :, 0] * panoptic_label_divisor + panoptic_pred_rgb[:, :, 1] * 256 + panoptic_pred_rgb[:, :, 2]
            
            panoptic_pred = panoptic_pred.astype(np.int32)
            seg_proto = camera_segmentation_utils.save_panoptic_label_to_proto(
                panoptic_pred,
                panoptic_label_divisor,
                context_name
            )
            seg_frame = metrics_pb2.CameraSegmentationFrame(
                camera_segmentation_label=seg_proto,
                context_name=context_name,
                frame_timestamp_micros=int(timestamp),
                camera_name=camera_left_to_right_order[i]
            )
            submission.predicted_segmentation_labels.frames.extend([seg_frame])
    

In [4]:

submit_type = "validation" # "testing"

context_with_issue = []
if(submit_type == "testing"):
    start_context = "10149575340910243572_2720_000_2740_000" # only append data starting from this context
    txtFile = "./DataLoader/2d_pvps_test_frames.txt"
elif(submit_type == 'validation'):
    start_context = "1024360143612057520_3580_000_3600_000"
    txtFile = "./DataLoader/2d_pvps_validation_frames.txt"
    
context_name_list, frames_dict = getFramesList(txtFile, start_context, context_with_issue)

save_folder = f"/media/kaiwenjon/Kevin-linux-dats/waymo/submission_{submit_type}"
for context_name in context_name_list:
    print("Context Name: ",context_name)
    timestamps = frames_dict[context_name]
    submission = _make_submission_proto()
    my_generate_predictions_for_one_test_shard(submission, context_name, timestamps)
    
    os.makedirs(save_folder, exist_ok=True)
    submission_basename = f"segment-{context_name}_camera_segmentation_submission.binproto"
    submission_file_path = os.path.join(save_folder, submission_basename)
    print(f'Saving predictions to {submission_file_path}...\n')
    f = open(submission_file_path, 'wb')
    f.write(submission.SerializeToString())
    f.close()

Context Name:  1024360143612057520_3580_000_3600_000
Saving predictions to /media/kaiwenjon/Kevin-linux-dats/waymo/submission_validation/segment-1024360143612057520_3580_000_3600_000_camera_segmentation_submission.binproto...

Context Name:  11048712972908676520_545_000_565_000
Saving predictions to /media/kaiwenjon/Kevin-linux-dats/waymo/submission_validation/segment-11048712972908676520_545_000_565_000_camera_segmentation_submission.binproto...

Context Name:  12831741023324393102_2673_230_2693_230
Saving predictions to /media/kaiwenjon/Kevin-linux-dats/waymo/submission_validation/segment-12831741023324393102_2673_230_2693_230_camera_segmentation_submission.binproto...

Context Name:  13982731384839979987_1680_000_1700_000
Saving predictions to /media/kaiwenjon/Kevin-linux-dats/waymo/submission_validation/segment-13982731384839979987_1680_000_1700_000_camera_segmentation_submission.binproto...

Context Name:  14081240615915270380_4399_000_4419_000
Saving predictions to /media/kaiwenj

In [6]:
gzFileName = f"submit_{submit_type}.tar.gz"
!tar czvf /media/kaiwenjon/Kevin-linux-dats/waymo/$gzFileName -C $save_folder .

./
./segment-8956556778987472864_3404_790_3424_790_camera_segmentation_submission.binproto
./segment-14081240615915270380_4399_000_4419_000_camera_segmentation_submission.binproto
./segment-5847910688643719375_180_000_200_000_camera_segmentation_submission.binproto
./segment-18024188333634186656_1566_600_1586_600_camera_segmentation_submission.binproto
./segment-13982731384839979987_1680_000_1700_000_camera_segmentation_submission.binproto
./segment-11048712972908676520_545_000_565_000_camera_segmentation_submission.binproto
./segment-17694030326265859208_2340_000_2360_000_camera_segmentation_submission.binproto
./segment-4759225533437988401_800_000_820_000_camera_segmentation_submission.binproto
./segment-1024360143612057520_3580_000_3600_000_camera_segmentation_submission.binproto
./segment-5372281728627437618_2005_000_2025_000_camera_segmentation_submission.binproto
./segment-6680764940003341232_2260_000_2280_000_camera_segmentation_submission.binproto
./segment-7119831293178745002_